## Import Library
Import all libraries required to run the functional genetic algorithm

In [ ]:
import os
import time
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

## City & Route Class

The City class to make it easier to define cities coordinate and calculate the distance between two cities.

The Route class to determine the route taken and calculate the total distance and fitness value

In [ ]:
class City:
    def __init__(self,name, x, y):
        self.x = x
        self.y = y
        self.name = name

    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((abs(self.x - city.x) ** 2) + (abs(self.y - city.y)** 2))
        return distance

    def __repr__(self):
        return "("+str(self.name)+")"

In [ ]:
%%capture cap
class Route:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness = 0.0

    def routeDistance(self):
        if self.distance == 0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None

                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]

                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance

    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

    def printCity(self):
        namaKota =[]
        for i in range(0,len(self.route)):
            namaKota.append(self.route[i].name)
        return "("+str(namaKota)+ ")"

## Genetic Algorithm Functions

Create several functions to perform each step in the genetic algorithm, such as:
- Create Route
- Initial Population
- Rank Routes
- Selection
- Check Tabulist
- Crossover
- Mutate
- Update Generation

### Create Route

In [ ]:
def createRoute(cityList):
    #Randomize city based on city in dataset to assign into route
    route = random.sample(cityList, len(cityList))
    return route

### Intialization the Population

In [ ]:
def initialPopulation(popSize, cityList):
    population = []
    # Create population based on population size
    for i in range(0, popSize):
      route = createRoute(cityList)
      while route in population:
        route = createRoute(cityList)
      population.append(createRoute(cityList))
    return population

### Rank Routes

In [ ]:
def rankRoutes(population):
    fitnessResults = []
    rankRoute = {}

    #Determine fitness each individual in population
    for i in range(0, len(population)):
        fitnessResults.append(Route(population[i]).routeFitness())

    #Sorting based on fitness value
    sortedFitness = sorted(fitnessResults, reverse=True)
    for i in range(0, len(sortedFitness)):
        for j in range(0, len(population)):
            if sortedFitness[i] == Route(population[j]).routeFitness():
                rankRoute[i] = population[j]

    return rankRoute

### Selection

In [ ]:
def selection(population):
    selection = []
    #Take first 2  the best individuals
    for i in range(0,2):
        selection.append(population[i])
        print("seleksi ke-" + str(i+1) + " " + str(Route(selection[i]).printCity()) + "\n" +
              " Jarak = " + str(Route(selection[i]).routeDistance()) + "\n" + " Fitness = " + str(
            Route(selection[i]).routeFitness()))
    return selection

### Tabulist

In [ ]:
def tabulist(selectionResults,tabulist):
    isTabulist1 = False
    isTabulist2 = False
    isTabulist = False

    #Check tabulist
    for j in range(0,len(tabulist)):
        if selectionResults[0] == tabulist[j]:
            isTabulist1 = True
        if selectionResults[1] == tabulist[j]:
            isTabulist2 = True

    if isTabulist1 == False:
        tabulist.append(selectionResults[0])
    if isTabulist2 == False:
        tabulist.append(selectionResults[1])

    if isTabulist1 == True and isTabulist2 == True:
        isTabulist = True

    return isTabulist,tabulist

### Crossover

In [ ]:
def crossover(parent1, parent2,tab):
    childP1 = []
    childP2 = []
    child = []

    # randomizes the gene number to be carried out crossover
    # if tabulis is true then the crossover result is the same as parent1 (best individual)
    if tab == True:
        geneA = 0
        geneB = len(parent1)-1
    else:
        geneA = random.randint(1, len(parent1)-1)
        geneB = random.randint(1, len(parent1)-1)
        # Gene A and Gene B do not have the same value
        while geneA == geneB:
            geneB = random.randint(1, len(parent1)-1)

    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)
    print("stargene = "+str(startGene+1)+", endGene = "+str(endGene+1))

    for i in range(startGene, endGene+1):
        childP1.append(parent1[i])

    for item in parent2:
        if item not in childP1:
            childP2.append(item)

    #Crossover Result
    idxChild1 = 0
    idxChild2 = 0
    for i in range(len(parent1)):
      if i >= startGene and i <= endGene:
        child.append(childP1[idxChild1])
        idxChild1 += 1
      else:
        child.append(childP2[idxChild2])
        idxChild2 += 1

    return child

### Mutate

In [ ]:
def mutate(individual):
    #randomizes the gene number to be exchanged
    swapped = random.randint(1, len(individual)-1)
    swapWith = random.randint(1, len(individual)-1)
    #swapped and swapWith cannot have the same value
    while swapped == swapWith:
        swapWith = random.randint(1, len(individual)-1)

    city1 = individual[swapped]
    city2 = individual[swapWith]

    print("swapped = "+str(swapped+1) + " , swap with = " + str(swapWith+1))

    #hasil mutasi
    individual[swapped] = city2
    individual[swapWith] = city1
    return individual

### Update Generation

In [ ]:
def updateGeneration(population,popsize,mutate):
    generasibaru =[]
    #combining populations with mutation results to create a new generation
    for i in range(0,popsize):
        generasibaru.append(population[i])

    #replace the individual with the smallest fitness (last index) with the mutation result if the mutation result is greater
    if Route(mutate).routeFitness() >= Route(generasibaru[-1]).routeFitness():
      generasibaru[popsize-1] =mutate

    for i in range(0,popsize):
        print("Generasi baru ke-" + str(i+1) + " " + str(Route(generasibaru[i]).printCity()) +
              " Jarak = " + str(Route(generasibaru[i]).routeDistance()) + " Fitness = " + str(
            Route(generasibaru[i]).routeFitness()))

    return generasibaru

### Implement The Genetica Algorithm

In [ ]:
def geneticAlgorithm(population, popSize, generations):
    tabulistResult = []
    finalPopulation = []
    pop = []

    # Create Population
    populasi = initialPopulation(popSize, population)

    # Sort each individual in population based on ranking
    rankPopulasi = rankRoutes(populasi)
    for i in range(0, popSize):
        pop.append(rankPopulasi[i])

    # Start genetic algorithm with the first generation
    print("=========================== GENERASI KE " + str(1) + "===============================================================>")
    for i in range(0,len(pop)):
        print("populasi ke-" + str(i+1) + " " + str(Route(pop[i]).printCity()) +
              " Jarak = " + str(Route(pop[i]).routeDistance()) + " Fitness = " + str(
            Route(pop[i]).routeFitness()))
    print("==========================================================================================>")

    bestDistance = 9999999999999
    progress =[]
    for i in range(0, generations):
        nextGeneration = []

        #Selection process
        selectionResults = selection(pop)

        #Check Tabulist
        tab, tabulistResult = tabulist(selectionResults, tabulistResult)

        #Crossover if the tabulist true
        crossoverResult = crossover(selectionResults[0], selectionResults[1], tab)
        if tab == False:
            print("Tabulist False")
            print("==========================================================================================>")
            print("hasil crossover = " + str(crossoverResult))
            print("==========================================================================================>")
        elif tab == True:
            print("Tabulist true")
            print("==========================================================================================>")
            print("seleksi 1 = " + str(crossoverResult))
            print("==========================================================================================>")
        print("TABULIST : ", tabulistResult, "\n")

        #Mutate Process
        children = mutate(crossoverResult)
        print("hasil mutasi = " + str(children) + " Jarak = " + str(Route(children).routeDistance()) + " Fitness = " + str(
            Route(children).routeFitness()))
        print("==========================================================================================>")

        #Update Generation
        nextGeneration = updateGeneration(pop, popSize, children)
        poptemp = nextGeneration

        # Sorting new generations based on fitness
        poptemp = rankRoutes(poptemp)
        pop = []
        for j in range(0, popSize):
            pop.append(poptemp[j])

        # Save the best individual in this generation
        tempbestroute = Route(pop[0]).routeDistance()
        progress.append(Route(pop[0]).routeDistance())
        if (bestDistance > tempbestroute):
            bestIndividu = Route(pop[0]).printCity()
            bestDistance = Route(pop[0]).routeDistance()
            bestFitness = Route(pop[0]).routeFitness()

        # Next Generation process
        finalPopulation = pop
        print("\n====================================== GENERASI KE " + str(
            i + 2) + "======================================>")
        for n in range(0, len(pop)):
            print("populasi ke-" + str(n+1) + " " + str(Route(pop[n]).printCity()) +
                " Jarak = " + str(Route(pop[n]).routeDistance()) + " Fitness = " + str(
                Route(pop[n]).routeFitness()))
        print("=========================================================================================>")


    print("BEST INDIVIDU = " + str(bestIndividu))
    print("BEST DISTANCE = " + str(bestDistance))
    print("BEST FITNESS = " + str(bestFitness))

    # Visualize the genetic algorithm result
    plt.figure(0)
    plt.plot(progress)
    plt.ylabel('Distance')
    plt.xlabel('Generation')
    plt.title('Genetica Algorithm Result')
    plt.savefig('algen_result.png')

    return finalPopulation

## Ant Colony Optimization Function

In [ ]:
def antColonyOptimization(city, iteration, nAnts, rho, alpha, beta, initialPheromne, routes):
    cityList = []
    for i in range(0, len(city)):
        cityList.append(City(name = city.iloc[i,0],x=city.iloc[i][1],y=city.iloc[i][2]))

    distances = np.zeros((len(cityList), len(cityList)))
    visibility = np.zeros((len(cityList), len(cityList)))

    for row in range(len(cityList)):
        for col in range (len(cityList)):
            distance = cityList[row].distance(cityList[col])
            distances[row, col] = distance
            visibility[row, col] = 1/distance  if distance != 0 else 0

    # Calculate total distance in Genetica algorithm result
    totalDistance = np.zeros((len(routes), 1)) # initiate total distance
    print(f"Genetica Algorithm Result: ")
    for i in range(len(routes)):
        routeStr = ""
        distance = 0
        for j in range(len(routes[i])-1):
            distance += routes[i][j].distance(routes[i][j+1])
            routeStr += " "+str(routes[i][j].name)
        routeStr += " "+str(routes[i][-1].name)
        print(f"Ant {i+1}: [{routeStr}] Distance: {Route(routes[i]).routeDistance()}")
        totalDistance[i] = distance

    # Inisialization Pheromne
    pheromne = initialPheromne * np.ones((len(cityList), len(cityList)))
    # Update pheromne with genetica algorithm result
    pheromne = (1 - rho) * pheromne # Evaporation
    for i in range(routes):
        delta = 1 / totalDistance[i][0] # Delta Pheromne
        for j in range(len(cityList)):
            pheromne[int(routes[i][j].name) - 1, int(routes[i][j+1].name) - 1] += delta
            pheromne[int(routes[i][j + 1].name) - 1, int(routes[i][j].name) - 1] += delta

    print("\nInitial Pheromne")
    print("--------------------------------")
    print("Initail City | Destination City | Pheromne")
    for row in range(len(cityList)):
        for col in range (row+1,len(cityList)):
            print(f"{row + 1}             | {col + 1}               | {pheromne[row, col]:.4f}")
    print("------------------------------------------\n")

    bestRoute = None
    bestDistance = float('inf')
    bestDistances = []

    # ACO Iteration
    for idx in range(iteration):
        print(f"=========================== Iteration {idx+1} ============================")
        antAndDistanceStr = ""

        routes = np.ones((nAnts, len(cityList)+1), dtype=int)
    
        # Randomize the first city each ants
        initialCitiesIdx = np.random.permutation(len(cityList))
        totalDistance = np.zeros((nAnts, 1))
        for i in range(nAnts):
            distance = 0
            routes[i, 0] = initialCitiesIdx[i] + 1 # Assign first city to routes
            visibilityTemp = np.array(visibility)

            for j in range(len(cityList)-1):

                # Calculate Probabilities
                currentLocation = int(routes[i, j] -1)
                visibilityTemp[:, currentLocation] = 0

                pFeature = np.power(pheromne[currentLocation, :], beta)
                vFeature = np.power(visibilityTemp[currentLocation, :], alpha)
                features = np.multiply(pFeature, vFeature)
                total = np.sum(features)
                probabilities = features/total

                print(f"Ant {i + 1}: {routes[i, :]}")
                print("---------------------")
                print("City  | Probability |")
                for k in range(len(cityList)):
                    print(f"{k + 1}     | {probabilities[k]:.4f}       |")
                print("---------------------")

                # Choose next city with highest probability
                nextCityIdx = np.argmax(probabilities)
                routes[i, j+1] = nextCityIdx + 1 # Add next city to route

                distance += distances[int(routes[i, j]) - 1, int(routes[i, j+1]) - 1]


            routes[i, -1] = routes[i, 0] # Back to first City
            print(f"Ant {i + 1}: {routes[i, :]}")

            # Calculate last city to first city
            distance += distances[int(routes[i, -2]) - 1, int(routes[i, -1]) - 1]
            totalDistance[i] = distance
            antAndDistanceStr += f"Ant {i+1}: {'-'.join(map(str, map(int, routes[i, :])))} | Distance = {totalDistance[i, 0]:.4f}\n"

            print("\n====================\n")

        print(f"Iteration {idx} Result: ")
        print(antAndDistanceStr)


        # Search the best routes
        distanceMinIdx = np.argmin(totalDistance)
        distanceMin = totalDistance[distanceMinIdx]
        if distanceMin < bestDistance:
            bestDistance = distanceMin
            bestRoute = routes[distanceMinIdx, :]

        bestDistances.append(bestDistance)

        # Update pheromne
        pheromne = (1 - rho) * pheromne # Evaporation
        for i in range(nAnts):
            delta = 1 / totalDistance[i][0] # Delta Pheromne
            for j in range(len(cityList)):
                pheromne[int(routes[i, j]) - 1, int(routes[i, j+1]) - 1] += delta
                pheromne[int(routes[i, j + 1]) - 1, int(routes[i, j]) - 1] += delta

        print("Update Pheromne")
        print("--------------------------------")
        print("Initail City | Destination City | New Pheromne")
        for i in range(len(cityList)):
            for j in range(i+1, len(cityList)):
                pheromneValue = pheromne[i, j]
                print(f"{i + 1}             | {j + 1}               | {pheromneValue:.4f}")
        print("------------------------------------------\n")


    print(f"The best routes: {'-'.join(map(str, map(int, bestRoute)))} | Total Distance = {bestDistance[0]:.4f}")

    # Ploting ACO Result
    plt.figure(1)
    plt.plot(range(1, iteration + 1), bestDistances)
    plt.xlabel('Iteration')
    plt.ylabel('Distance')
    plt.title('Ant Colony Optimization Result')
    plt.savefig('aco_result.png')

## Main

There are several variables that you must fill in first:

- datasetPath (fielname the dataset)

Genertic Algorithm Parameters:
- populationSize (number of population)
- generation (number of generation)

Ant Colony Optimization Algorithm:
- iteration (number of iteration)
- nAnts (number of ants)
- rho (number of evaporation pheromne)
- alpha
- beta
- initialPheromne

In [59]:
%%capture cap --no-stderr

if os.path.exists('genetica_and_aco_result.txt'):
    os.remove('genetica_and_aco_result.txt')
if os.path.exists('aco_result.png'):
    os.remove('aco_result.png')
if os.path.exists('algen_result.png'):
    os.remove('algen_result.png')

datasetPath = "lin318.csv"

# Algen Parameters
populationSize = 10
generation = 10

# ACO Parameters
iteration = 3
nAnts = 20
rho = 0.5
alpha = 1
beta = 1
initialPheromne = 10

start_time = time.time()

city = pd.read_csv(datasetPath, header=None , sep=' ')
cityList = []
for i in range(0,len(city)):
    cityList.append(City(name = city.iloc[i,0],x=city.iloc[i][1],y=city.iloc[i][2]))

# Start Genetic Algorithm process
print("\n====================================== Genetica Algorithm ======================================\n")
print(f"Population Size: {str(populationSize)}")
print(f"Generation: {str(generation)}")
print()
result = geneticAlgorithm(population=cityList, popSize=populationSize, generations=generation)

end_algen_time = time.time()
algo_time = end_algen_time - start_time

# add first city to last index in each route
newPop = []
for r in result:
  firstCity = r[0]
  newRoute = []
  for r2 in r:
    newRoute.append(r2)
  newRoute.append(firstCity)
  newPop.append(newRoute)

# Start Ant Colony Optimization Algorithm process
print("\n\n====================================== Ant Colony Optimization ======================================\n")
print(f"Iteration: {str(iteration)}")
print(f"Ants: {str(nAnts)}")
print(f"Rho: {str(rho)}")
print(f"Alpha: {str(alpha)}")
print(f"Beta: {str(beta)}")
print(f"Initial Pheromne : {str(initialPheromne)}")
print()



antColonyOptimization(city, iteration, nAnts, rho, alpha, beta, initialPheromne, newPop)
aco_time = time.time() - end_algen_time
executionTime = time.time() - start_time

print("GENETIC ALGORITHM TIME: {hour:.4f} hour, {minutes:.4f} minutes, {seconds:.4f} seconds".format(hour = algo_time/3600, minutes = algo_time/60, seconds = algo_time))
print("ACO ALGORITHM TIME: {hour:.4f} hour, {minutes:.4f} minutes, {seconds:.4f} seconds".format(hour = aco_time/3600, minutes = aco_time/60, seconds = aco_time))
print("EXECUTION TIME =: {hour:.4f} hour, {minutes:.4f} minutes, {seconds:.4f} seconds".format(hour = executionTime/3600, minutes = executionTime/60, seconds = executionTime))


KeyboardInterrupt: 

In [ ]:
with open('genetica_and_aco_result.txt', 'w') as f:
    f.write(cap.stdout)